In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
world_cup = pd.read_csv('World Cup 2018 Dataset.csv')
world_cup = world_cup.replace({"IRAN": "Iran", 
                               "Costarica": "Costa Rica", 
                               "Porugal": "Portugal", 
                               "Columbia": "Colombia", 
                               "Korea" : "Korea Republic"})
world_cup = world_cup.loc[:, ['Team', 'Group', 'First match \nagainst', 'Second match\n against', 'Third match\n against']]
world_cup.dropna(inplace=True)

world_cup = world_cup.set_index('Team')

In [3]:
spi_2014 = pd.read_csv('2014_forecasts.csv')

spi_2014

,date,time,country,country_code,group_letter,spi,spi_offense,spi_defense,group,sixteen,quarter_finals,semi_finals,final,champion
0,10 June,14:00,Algeria,ALG,H,63.43,1.1208,1.1636,0.0631,0.2032,0.038517,0.007996,0.001021,0.000126
1,10 June,14:00,Argentina,ARG,F,90.00,2.8541,0.4494,0.7350,0.9279,0.669904,0.468159,0.281758,0.127799
2,10 June,14:00,Australia,AUS,B,69.45,1.6395,1.2349,0.0151,0.0762,0.009646,0.002943,0.000671,0.000093
3,10 June,14:00,Belgium,BEL,H,81.97,2.1373,0.7410,0.4781,0.7688,0.351536,0.148459,0.054136,0.014904
4,10 June,14:00,Bosnia and Herzegovina,BIH,F,80.31,2.3113,0.9861,0.1599,0.5589,0.261950,0.112098,0.031611,0.008964
5,10 June,14:00,Brazil,BRA,A,91.83,3.3686,0.5175,0.9500,0.9940,0.804479,0.697590,0.565823,0.453437
6,10 June,14:00,Chile,CHI,B,86.72,2.7113,0.7043,0.3392,0.6951,0.324846,0.205733,0.110337,0.041899
7,10 June,14:00,Ivory Coast,CIV,C,78.90,2.2467,1.0480,0.2317,0.5287,0.243748,0.067887,0.025096,0.006044
8,10 June,14:00,Cameroon,CMR,A,71.33,1.4798,0.9904,0.0079,0.2272,0.050187,0.016983,0.004380,0.000585
9,10 June,14:00,Colombia,COL,C,85.82,2.1813,0.4528,0.5156,0.7794,0.457763,0.149779,0.073364,0.028030


In [4]:
spi_2014 = spi_2014[['date', 'country', 'spi', 'spi_offense', 'spi_defense']]
spi_2014['date'] = pd.to_datetime(spi_2014['date'], format="%d %B")
spi_2014['date'] = spi_2014['date'].dt.strftime('%d-%b')

spi_2014 = spi_2014.groupby(['country', 'date']).agg('mean')

In [5]:
matches = pd.read_csv('2014_scores.csv')
matches.head()

,match_id,match_date,match_time,home_country,home_code,home_goals,away_country,away_code,away_goals,win_country,win_code
0,1,12 June,21:00,Brazil,BRA,3,Croatia,CRO,1,Brazil,BRA
1,2,13 June,17:00,Mexico,MEX,1,Cameroon,CMR,0,Mexico,MEX
2,3,13 June,20:00,Spain,ESP,1,Netherlands,NED,5,Netherlands,NED
3,4,13 June,23:00,Chile,CHI,3,Australia,AUS,1,Chile,CHI
4,5,14 June,17:00,Colombia,COL,3,Greece,GRE,0,Colombia,COL


In [6]:
matches.rename(columns={'match_date': 'date'}, inplace=True)
matches['date'] = pd.to_datetime(matches['date'], format="%d %B")
matches['date'] = matches['date'].dt.strftime('%d-%b')

In [7]:
matches_home = matches[['match_id', 'date', 'home_country', 'home_goals']]

matches_home.rename(columns={'home_country': 'country'}, inplace=True)
matches_home.set_index(['country', 'date'], inplace=True)

matches_home = pd.concat([matches_home, spi_2014], axis=1, join='inner')
matches_home.rename(columns={'spi': 'home_spi',
                            'spi_offense': 'home_spi_off',
                            'spi_defense': 'home_spi_def'}, inplace=True)

matches_home.reset_index(inplace=True)
matches_home.set_index('match_id', inplace=True)

In [8]:
matches_away = matches[['match_id', 'date', 'away_country', 'away_goals']]

matches_away.rename(columns={'away_country': 'country'}, inplace=True)
matches_away.set_index(['country', 'date'], inplace=True)

matches_away = pd.concat([matches_away, spi_2014], axis=1, join='inner')
matches_away.rename(columns={'spi': 'away_spi',
                            'spi_offense': 'away_spi_off',
                            'spi_defense': 'away_spi_def'}, inplace=True)

matches_away.reset_index(inplace=True)
matches_away.set_index('match_id', inplace=True)

In [9]:
matches = pd.concat([matches_home, matches_away], axis=1, join='inner')
matches.sort_index(inplace=True)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
home_predictors = matches[[
 #'home_spi',
 'home_spi_off',
 'home_spi_def',
 #'away_spi',
 'away_spi_off',
 'away_spi_def']]

home_goals = matches[['home_goals']]

lr_home = LinearRegression()

lr_home.fit(home_predictors, home_goals)

print("R2 score: {}".format(lr_home.score(home_predictors, home_goals)))
print("Coefficients: {}".format(lr_home.coef_))
print("Intercept: {}".format(lr_home.intercept_))

matches['pred_home_score'] = 0

for i in range(lr_home.coef_.shape[1]):
    matches['pred_home_score'] += lr_home.coef_[0][i] * home_predictors.iloc[:,i]
    
matches['pred_home_score'] += lr_home.intercept_[0]

R2 score: 0.1651231931525241
Coefficients: [[ 0.4308677  -0.18288087 -0.23187745  1.18347326]]
Intercept: [0.00967951]


In [13]:
away_predictors = matches[[
 #'home_spi',
 'home_spi_off',
 'home_spi_def',
 #'away_spi',
 'away_spi_off',
 'away_spi_def']]

away_goals = matches[['away_goals']]

lr_away = LinearRegression()

lr_away.fit(away_predictors, away_goals)

print("R2 score: {}".format(lr_away.score(away_predictors, away_goals)))
print("Coefficients: {}".format(lr_away.coef_))
print("Intercept: {}".format(lr_away.intercept_))

matches['pred_away_score'] = 0

for i in range(lr_away.coef_.shape[1]):
    matches['pred_away_score'] += lr_away.coef_[0][i] * away_predictors.iloc[:,i]
    
matches['pred_away_score'] += lr_away.intercept_[0]

matches[['country', 'home_goals', 'pred_home_score', 'away_goals', 'pred_away_score']]

R2 score: 0.18188885872258476
Coefficients: [[-0.23536562  1.28052301  0.87692167 -0.46668484]]
Intercept: [-0.54055543]


,country,country,home_goals,pred_home_score,away_goals,pred_away_score
match_id,,,,,,
1,Brazil,Croatia,3,2.045393,1,0.439924
2,Mexico,Cameroon,1,1.381619,0,0.812369
3,Spain,Netherlands,1,1.516635,5,1.046532
4,Chile,Australia,3,2.126142,1,0.585015
5,Colombia,Greece,3,1.192751,0,0.448089
6,Ivory Coast,Japan,2,1.881731,1,1.518890
7,Uruguay,Costa Rica,1,1.362475,3,0.668843
8,England,Italy,1,1.305715,2,1.270737
9,Switzerland,Ecuador,2,1.118147,1,1.599584


In [45]:
spi_2018 = pd.read_csv('2018_forecasts.csv')
spi_2018 = spi_2018.replace({'South Korea': 'Korea Republic'})
spi_2018.head()

,forecast_timestamp,team,group,spi,global_o,global_d,sim_wins,sim_ties,sim_losses,sim_goal_diff,...,group_1,group_2,group_3,group_4,make_round_of_16,make_quarters,make_semis,make_final,win_league,timestamp
0,2018-06-17 19:56:22 UTC,Brazil,E,92.39264,3.05704,0.31349,1.41355,1.40014,0.18631,2.94050,...,0.54856,0.30233,0.12325,0.02586,0.85089,0.61093,0.42513,0.27652,0.17721,2018-06-17 19:58:10 UTC
1,2018-06-17 19:56:22 UTC,Spain,B,91.49942,3.32662,0.50111,1.43941,1.38775,0.17284,3.07257,...,0.57997,0.29632,0.08798,0.03573,0.87629,0.67363,0.45874,0.28964,0.17413,2018-06-17 19:58:10 UTC
2,2018-06-17 19:56:22 UTC,Germany,F,89.87194,3.13775,0.52709,1.40459,0.36875,1.22666,2.06499,...,0.22977,0.42564,0.26516,0.07943,0.65541,0.38744,0.25597,0.15576,0.08982,2018-06-17 19:58:10 UTC
3,2018-06-17 19:56:22 UTC,France,C,86.53100,2.82202,0.57358,2.10888,0.50183,0.38929,2.63732,...,0.64157,0.25032,0.09842,0.00969,0.89189,0.53909,0.30055,0.15237,0.07813,2018-06-17 19:58:10 UTC
4,2018-06-17 19:56:22 UTC,England,G,83.72824,2.34392,0.45752,1.77145,0.71491,0.51364,2.86514,...,0.43855,0.41108,0.11597,0.03440,0.84963,0.57242,0.29001,0.14897,0.07281,2018-06-17 19:58:10 UTC


In [46]:
spi_2018 = spi_2018[[
 'team',
 'group',
 #'spi',
 'global_o',
 'global_d'
]]

spi_2018 = spi_2018.groupby('team').agg('mean')

spi_2018.rename(columns={'global_o': 'spi_off',
                         'global_d': 'spi_def'}, inplace=True)

In [60]:
spi_2018

,spi_off,spi_def
team,,
Argentina,2.580760,0.403535
Australia,1.796310,1.032395
Belgium,2.810960,0.632620
Brazil,3.082900,0.309210
Colombia,2.182400,0.512460
Costa Rica,1.712305,0.849855
Croatia,2.300645,0.574655
Denmark,2.117405,0.737905
Egypt,1.578980,0.853635


In [30]:
world_cup

,Group,First match against,Second match against,Third match against
Team,,,,
Russia,A,Saudi Arabia,Egypt,Uruguay
Saudi Arabia,A,Russia,Uruguay,Egypt
Egypt,A,Uruguay,Russia,Saudi Arabia
Uruguay,A,Egypt,Saudi Arabia,Russia
Portugal,B,Spain,Morocco,Iran
Spain,B,Portugal,Iran,Morocco
Morocco,B,Iran,Portugal,Spain
Iran,B,Morocco,Spain,Portugal
France,C,Australia,Peru,Denmark


In [51]:
from itertools import combinations

opponents = ['First match \nagainst', 'Second match\n against', 'Third match\n against']

world_cup['points'] = 0

for group in set(world_cup['Group']):
    print('___Starting group {}:___'.format(group))
    print("\n")
    for home, away in combinations(world_cup.query('Group == "{}"'.format(group)).index, 2):
        
        home_score =    lr_home.coef_[0][0] * spi_2018.loc[home, 'spi_off'] +\
                        lr_home.coef_[0][1] * spi_2018.loc[home, 'spi_def'] +\
                        lr_home.coef_[0][2] * spi_2018.loc[away, 'spi_off'] +\
                        lr_home.coef_[0][3] * spi_2018.loc[away, 'spi_def'] +\
                        lr_home.intercept_[0]
        
        away_score =    lr_away.coef_[0][0] * spi_2018.loc[home, 'spi_off'] +\
                        lr_away.coef_[0][1] * spi_2018.loc[home, 'spi_def'] +\
                        lr_away.coef_[0][2] * spi_2018.loc[away, 'spi_off'] +\
                        lr_away.coef_[0][3] * spi_2018.loc[away, 'spi_def'] +\
                        lr_away.intercept_[0]
        
        if home_score >= away_score + 0.5:
            
            world_cup.loc[home, 'points'] += 3
            
            if round(home_score, 0) == round(away_score, 0):
                print("{} wins {}: {:0.0f} - {:0.0f}".format(home, away, home_score+0.5, away_score-0.5)) 
            else:    
                print("{} wins {}: {:0.0f} - {:0.0f}".format(home, away, home_score, away_score)) 
             
        elif away_score >= home_score +0.5: 
            
            world_cup.loc[away, 'points'] += 3
            
            if round(home_score, 0) == round(away_score, 0):
                print("{} loses {}: {:0.0f} - {:0.0f}".format(home, away, home_score-0.5, away_score+0.5))
            else:    
                print("{} loses {}: {:0.0f} - {:0.0f}".format(home, away, home_score, away_score))
            
        else:
            world_cup.loc[home, 'points'] += 1
            world_cup.loc[away, 'points'] += 1
            
            print("{} draws {}: {:0.0f} - {:0.0f}".format(home, away, (home_score+away_score)/2, (home_score+away_score)/2))
    
    print("\n")
      

___Starting group C:___


France wins Australia: 2 - 1
France wins Peru: 2 - 1
France draws Denmark: 1 - 1
Australia loses Peru: 1 - 2
Australia loses Denmark: 1 - 2
Peru draws Denmark: 1 - 1


___Starting group E:___


Brazil wins Switzerland: 2 - 1
Brazil wins Costa Rica: 2 - 0
Brazil wins Serbia: 2 - 1
Switzerland wins Costa Rica: 2 - 0
Switzerland draws Serbia: 1 - 1
Costa Rica loses Serbia: 1 - 2


___Starting group G:___


Belgium wins Panama: 2 - 0
Belgium wins Tunisia: 2 - 1
Belgium draws England: 1 - 1
Panama draws Tunisia: 1 - 1
Panama loses England: 0 - 2
Tunisia loses England: 1 - 2


___Starting group A:___


Russia wins Saudi Arabia: 2 - 1
Russia draws Egypt: 1 - 1
Russia loses Uruguay: 1 - 2
Saudi Arabia loses Egypt: 1 - 2
Saudi Arabia loses Uruguay: 1 - 2
Egypt loses Uruguay: 1 - 2


___Starting group B:___


Portugal loses Spain: 1 - 2
Portugal draws Morocco: 1 - 1
Portugal wins Iran: 2 - 0
Spain wins Morocco: 2 - 1
Spain wins Iran: 2 - 0
Morocco draws Iran: 1 - 1


__

In [58]:
world_cup = world_cup.sort_values(['Group', 'points'], ascending=False).groupby('Group')[['Group', 'points']].head(2)

pairing = [0,3,4,7,8,11,12,15,1,2,5,6,9,10,13,14]

next_round_wc = world_cup.reset_index()
next_round_wc = next_round_wc.loc[pairing]
next_round_wc = next_round_wc.set_index('Team')

In [59]:
finals = ['round_of_16', 'quarterfinal', 'semifinal', 'final']

labels = list()
odds = list()

for f in finals:
    print("___Starting of the {}___".format(f))
    iterations = int(len(next_round_wc) / 2)
    winners = []

    for i in range(iterations):
        home = next_round_wc.index[i*2]
        away = next_round_wc.index[i*2+1]
        
        home_score =    lr_home.coef_[0][0] * spi_2018.loc[home, 'spi_off'] +\
                        lr_home.coef_[0][1] * spi_2018.loc[home, 'spi_def'] +\
                        lr_home.coef_[0][2] * spi_2018.loc[away, 'spi_off'] +\
                        lr_home.coef_[0][3] * spi_2018.loc[away, 'spi_def'] +\
                        lr_home.intercept_[0]
        
        away_score =    lr_away.coef_[0][0] * spi_2018.loc[home, 'spi_off'] +\
                        lr_away.coef_[0][1] * spi_2018.loc[home, 'spi_def'] +\
                        lr_away.coef_[0][2] * spi_2018.loc[away, 'spi_off'] +\
                        lr_away.coef_[0][3] * spi_2018.loc[away, 'spi_def'] +\
                        lr_away.intercept_[0]
        
        print(home_score, away_score)
        
        if home_score > away_score:
            if round(home_score, 0) == round(away_score, 0):
                print("{} wins {}: {:0.0f} - {:0.0f}".format(home, away, home_score+0.5, away_score-0.5)) 
            else:    
                print("{} wins {}: {:0.0f} - {:0.0f}".format(home, away, home_score, away_score)) 
             
            winners.append(home)
            
        elif home_score < away_score: 
            if round(home_score, 0) == round(away_score, 0):
                print("{} loses {}: {:0.0f} - {:0.0f}".format(home, away, home_score-0.5, away_score+0.5))
            else:    
                print("{} loses {}: {:0.0f} - {:0.0f}".format(home, away, home_score, away_score)) 
            
            winners.append(away)
            
    next_round_wc = next_round_wc.loc[winners]
    print("\n") 

___Starting of the round_of_16___
0.8542465379639028 1.4439160546010632
Colombia loses England: 0 - 2
1.5563552969555297 1.0807904820377034
Germany wins Switzerland: 2 - 1
1.4301591663418478 0.8811874974332812
Argentina wins Denmark: 2 - 0
1.799646775299053 0.6762830575537908
Spain wins Russia: 2 - 1
0.8430546967066889 2.2187195928567736
Japan loses Belgium: 1 - 2
0.4679680210339899 2.403689550532261
Mexico loses Brazil: 0 - 2
0.9106502679643399 1.8896876861314746
Croatia loses France: 1 - 2
1.164937808042778 1.2621710369379828
Portugal loses Uruguay: 1 - 2


___Starting of the quarterfinal___
0.8181843018255248 2.042304126714564
England loses Germany: 1 - 2
0.8594492494728686 2.010703872143906
Argentina loses Spain: 1 - 2
0.7562240548507186 2.167083889215169
Belgium loses Brazil: 1 - 2
1.324669251326607 1.2353581753677583
France wins Uruguay: 2 - 1


___Starting of the semifinal___
1.0979050645926864 2.0234539068617843
Germany loses Spain: 1 - 2
1.2962434939676026 1.3656633221718004
B